In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [4]:
train = pd.read_json('C:/Users/HOME/Desktop/수DA쟁이/배, 빙하 분류하기/Data/train.json')
target_train=train['is_iceberg']
test = pd.read_json("C:/Users/HOME/Desktop/수DA쟁이/배, 빙하 분류하기/Data/test.json")

In [5]:
train.head()

,id,band_1,band_2,inc_angle,is_iceberg
0,dfd5f913,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",43.9239,0
1,e25388fd,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",38.1562,0
2,58b2aaa0,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",45.2859,1
3,4cfc3a18,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",43.8306,0
4,271f93f4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",35.6256,0


In [6]:
target_train.head()

0    0
1    0
2    1
3    0
4    0
Name: is_iceberg, dtype: int64

In [7]:
test.head()

,id,band_1,band_2,inc_angle
0,5941774d,"[-15.863251, -15.201077, -17.887735, -19.17248...","[-21.629612, -21.142353, -23.908337, -28.34524...",34.966400
1,4023181e,"[-26.058969497680664, -26.058969497680664, -26...","[-25.754207611083984, -25.754207611083984, -25...",32.615072
2,b20200e4,"[-14.14109992980957, -15.064241409301758, -17....","[-14.74563980102539, -14.590410232543945, -14....",37.505433
3,e7f018bb,"[-12.167478, -13.706167, -16.54837, -13.572674...","[-24.32222, -26.375538, -24.096739, -23.8769, ...",34.473900
4,4371c8c3,"[-23.37459373474121, -26.02718162536621, -28.1...","[-25.72234344482422, -27.011577606201172, -23....",43.918874


In [8]:
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')

### 결측치 채우기

결측값을 앞 방향으로 채우기 : fillna(method='ffill' or 'pad')<br>
결측값을 뒷 방향으로 채우기 : fillna.(method='bfill' or 'backfill')

In [9]:
train['inc_angle']=train['inc_angle'].fillna(method='pad')

In [10]:
X_angle=train['inc_angle'] # train_x값
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle'] # test_x값

### train, test 데이터 정규화

In [15]:
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75)
                   for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) 
                   for band in train["band_2"]])
X_band_3=(X_band_1+X_band_2)/2
#X_band_3=np.array([np.full((75, 75), angel).astype(np.float32) 
#for angel in train["inc_angle"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis]
                          , X_band_2[:, :, :, np.newaxis]
                         , X_band_3[:, :, :, np.newaxis]], axis=-1)


In [15]:
X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) 
                        for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) 
                        for band in test["band_2"]])
X_band_test_3=(X_band_test_1+X_band_test_2)/2
#X_band_test_3=np.array([np.full((75, 75), angel).astype(np.float32) 
#for angel in test["inc_angle"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , X_band_test_3[:, :, :, np.newaxis]], axis=-1)

In [19]:
# 필요한 패키지 개많네
from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input	

In [41]:
#Data Aug for multi-input 다중입력을 통한 이미지 확대?
from keras.preprocessing.image import ImageDataGenerator
batch_size=64

### 이미지 변환 정의

In [ ]:
# horizontal_flip, rescale, shear_range,zoom_range를 설정해줌으로써 해당 데이터의
#전처리를 어떻게 할지 결정
# horizontal_flip : Boolean. Randomly flip inputs horizontally.(가로 무작위 뒤집기)




gen = ImageDataGenerator(horizontal_flip = True, # 가로 무작위 뒤집기
                         vertical_flip = True, # 수직 방향 뒤집기
                         width_shift_range = 0., # 수평방향 범위내에서 이미지 이동
                         height_shift_range = 0., #수직방향 범위내에서 이미지 이동
                         channel_shift_range=0,
                         zoom_range = 0.2,  # 확대축소 범위 내에서 확대축소
                         rotation_range = 10) # 지정 각도 내에서 원본 이미지 회전

### 두 생성기 병합 함수

In [21]:
#y배열과 각도 배열에대해 동일한 랜덤 시드를 가진 정확히 동일한 생성기를 사용합니다.
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert 배열은 동일하다 - 
            #안심할 수 있지만 훈련속도가 느려짐
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

### 마지막으로 생성기 생성

In [23]:
#early stopping = epoch를 많이 돌린 후 특정 시점에서 멈추게 하기
#val_loss : validation set의 loss를 모니터링한다는 의미
#patience는 성능이 증가하지 않는 epoch를 몇 번이나 허용할 것인가를 의미
def get_callbacks(filepath, patience=2):
   # performance를 정의하고 최소화하는 es
   es = EarlyStopping('val_loss', patience=10, mode="min")
   #가장 좋은 모델 저장
   msave = ModelCheckpoint(filepath, save_best_only=True)
   return [es, msave]

### 모델 정의

In [42]:
def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    #사전 훈련된 VGG16모델 사용
    # 로딩할 가중치 =imagenet
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
    # block5_pool : 물체 특성을 뽑는 층까지만 떼서 쓰기 위한 코드
    x = base_model.get_layer('block5_pool').output
    

    x = GlobalMaxPooling2D()(x)
    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(input=[base_model.input, input_2], output=predictions)
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model


### kfold CV 정의

In [43]:
#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle, X_test):
    K=3
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        galaxyModel.fit_generator(gen_flow,
                                    steps_per_epoch=24,
                                    epochs=100,
                                    shuffle=True,
                                    verbose=1,
                                    validation_data=([X_holdout,X_angle_hold], Y_holdout),
                                    callbacks=callbacks)
        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

        y_test_pred_log=y_test_pred_log/K
        y_train_pred_log=y_train_pred_log/K

        print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
        print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
        return y_test_pred_log

In [44]:
#원래 다르게 오류가 나야하는데 ㅠㅠ
preds=myAngleCV(X_train, X_angle, X_test)

NameError: name 'X_test' is not defined

In [48]:
#Submission for each day.
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=preds
submission.to_csv('sub.csv', index=False)

NameError: name 'preds' is not defined